# Reachability Analysis for Split Computing Neural Networks

## 1. Import Packages

In [1]:
begin
	import Pkg
	Pkg.activate("..")
	Pkg.instantiate()
	
    push!(LOAD_PATH, "$(@__DIR__)/../src")
    
    using Plots
    using NoisyReach
    using Distributions
    using Experiment
    using QuadGK
    using ControlSystemsBase
    using LinearAlgebra
    using ReachabilityAnalysis
end

  Activating project at `~/NoisyReach.jl`


┌ Warning: The project dependencies or compat requirements have changed since the manifest was last resolved.
│ It is recommended to `Pkg.resolve()` or consider `Pkg.update()` if necessary.
└ @ Pkg.API /Users/tingan/.julia/juliaup/julia-1.10.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.10/Pkg/src/API.jl:1807


## 2. Validation for Integral function

In [2]:
#const Ts = 0.01
#const Dc = 0.005
#sys = benchmarks[:F1]

In [3]:
#sys_aug_ = let
#    ϕ = ℯ^(Ts * sys.A)
#    Γ₁ = matrix_integral(sys.A, sys.B, Dc, Ts)
#    Γ₀ = matrix_integral(sys.A, sys.B, 0.0, Ts - Dc)
#    ϕ_aug = [ϕ Γ₁; 0 0 0]
#    Γ_aug = [Γ₀; I]
#    C_aug = [sys.C 0]
#    ss(ϕ_aug, Γ_aug, C_aug, sys.D, Ts)
#end

In [4]:
#sys_aug = c2d(sys, Ts) * delay(Dc, Ts)
#sys_aug = c2d(sys, Ts) * thiran(Dc, Ts)

## 3. Physical System and Neural Networks Uncertainty Setup

In [5]:
sys = benchmarks[:F1]
const period = 0.02
const period_c = 0.001
const Dc1 =    0.005
const Dc2 =    period

0.02

In [6]:
sys_ideal = c2d(sys, period_c)
K_ideal = lqr(ControlSystemsBase.Discrete, sys_ideal.A, sys_ideal.B, I, I)

1×2 Matrix{Float64}:
 0.987398  1.27555

In [7]:
sys_aug = let
    ϕ = ℯ^(period * sys.A)
    Γ2 = matrix_integral(sys.A, sys.B, 0.0, period - Dc2)
    Γ1 = matrix_integral(sys.A, sys.B, 0.0, Dc2 - Dc1)
    Γ3 = matrix_integral(sys.A, sys.B, period - Dc2, period - Dc2+Dc1)
    ϕ_aug = [ϕ Γ3; 0 0 0]
    Γ_aug = [Γ1 Γ2; 0 I]
    C_aug = [sys.C 0]
    D_aug = [sys.D 0]
    ss(ϕ_aug, Γ_aug, C_aug, D_aug, period)
end
K = lqr(ControlSystemsBase.Discrete, sys_aug.A, sys_aug.B, I, I)

2×3 Matrix{Float64}:
 0.809378   1.17075   0.106169
 0.0707289  0.135018  0.0124974

In [8]:
σ1 = 0.3
σ2 = 0.2
μ = 0.
H = 1000
H_c= floor(Int, H * period / period_c)
x0 = 1.
u1_0 = 0.
u2_0 = 0.
u0 = 0.
n = 1
z0 = [fill(x0, size(sys.A, 1)); u2_0]
x0 = fill(x0, size(sys.A, 1))

2-element Vector{Float64}:
 1.0
 1.0

## 4. Calculate and Plot Reachable Trajectories

In [9]:
all_trajectories_z = []
all_trajectories_u = []
N = 10000
for i in 1:N
    z, u = evolve(sys_aug.A, sys_aug.B, K, H, z0, u1_0, u2_0, σ1, σ2, μ, n)
    push!(all_trajectories_z, z)
    push!(all_trajectories_u, u)
end
z_ideal, u_ideal = ideal_evolve(sys_ideal.A, sys_ideal.B, K_ideal, H_c, x0, u0)

([[1.0, 1.0], [1.0065, 1.0], [1.0128548141216618, 0.9553274220498026], [1.0189225005649245, 0.9116530220310246], [1.024709484167767, 0.8689573173051378], [1.030222064315044, 0.8272211895492936], [1.035466417284732, 0.7864258780470461], [1.0404485985509664, 0.7465529731019375], [1.0451745450446557, 0.707584409571698], [1.0496500773724566, 0.6695024605208595]  …  [1.0670426686916479e-60, -1.1404199619260659e-60], [1.0596555978753784e-60, -1.1325249046184094e-60], [1.05231966729637e-60, -1.1246845043072736e-60], [1.045034522913903e-60, -1.116898382605631e-60], [1.0377998131382603e-60, -1.1091661637460036e-60], [1.0306151888137583e-60, -1.1014874745623291e-60], [1.0234803032018968e-60, -1.0938619444719508e-60], [1.0163948119646253e-60, -1.0862892054577327e-60], [1.0093583731477242e-60, -1.0787688920502991e-60], [1.0023706471643019e-60, -1.0713006413103964e-60]], [[0.0], [-2.2693669598700295], [-2.2186595209539215], [-2.168941800075048], [-2.1201952899968886], [-2.0724018243141704], [-2.025

In [10]:
const xlim = 2
const ylim = 2
#Plots.scalefontsizes(0.1)

2

In [11]:
traj_plot = plot(
    xlabel = "\$x_1\$",
    ylabel = "\$x_2\$",
    xguidefont = font(24),   # make x-axis label bigger
    yguidefont = font(24),   # make y-axis label bigger
    xtickfont = font(16),    # optional: make tick labels bigger
    ytickfont = font(16)
)
for trajectory in all_trajectories_z
    x_z = [point[1] for point in trajectory]
    y_z = [point[2] for point in trajectory]
    
    plot!(x_z, y_z, xlim=(0, xlim), ylim=(-ylim, ylim), label="", linecolor=:lightgray, linewidth=1)#, marker=:circle, markercolor=:yellow, markersize=2)
end
x_z_ideal = [point[1] for point in z_ideal]
y_z_ideal = [point[2] for point in z_ideal]
plot!(x_z_ideal, y_z_ideal, xlim=(0, xlim+0.2), ylim=(-ylim, ylim), label="", linecolor=:black, linewidth=2)
savefig(traj_plot, "edge_only.png")

"/Users/tingan/NoisyReach.jl/experiments/edge_only.png"

In [12]:
dev_trajs = [[(z_ideal[1+(k-1)*floor(Int, period / period_c)] .- all_trajectories_z[i][k][1:2]) for k in 1:H] for i in 1:N]
dev₁_trajs = [[dev[1] for dev in dev_traj] for dev_traj in dev_trajs]

max_max_x₁ = maximum(xd -> maximum(xd -> abs(xd[1]), xd), all_trajectories_z)

mean_mean_dev₁ = mean([mean(abs.(dev₁_traj)) for dev₁_traj in dev₁_trajs])
max_max_dev₁ = maximum([maximum(abs.(dev₁_traj)) for dev₁_traj in dev₁_trajs])

std_rmse_dev₁ = std([std(dev₁_traj) for dev₁_traj in dev₁_trajs])
max_rmse_dev₁ = maximum([std(dev₁_traj) for dev₁_traj in dev₁_trajs])
mean_rmse_dev₁ = mean([std(dev₁_traj) for dev₁_traj in dev₁_trajs])

println("Std: ", std_rmse_dev₁)
println("Max: ", max_rmse_dev₁)
println("Mean: ", mean_rmse_dev₁)

Std: 0.006833365548400954


Max: 0.05915939124667274
Mean: 0.029354515844199625
